### Import libraries

In [7]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
from skimage import io
import sys
import seaborn as sns
from scipy import spatial
import warnings

### Parameters

In [2]:
single_frame = True

### Import data (AFT track results and merged Tracks)

In [5]:
csv_folder = "/Users/k1801626/OneDrive - King's College London/data/JP/AFT_track/"

In [9]:
file_name_pattern = '*/*_Spots_AFT_single_frame_'+str(single_frame)+'.csv'
file_list = glob.glob(csv_folder+file_name_pattern)

print(*file_list, sep="\n")

/Users/k1801626/OneDrive - King's College London/data/JP/AFT_track/siCCT8_si8/siCCT8_si8_Spots_AFT_single_frame_True.csv
/Users/k1801626/OneDrive - King's College London/data/JP/AFT_track/siCTRL/siCTRL_Spots_AFT_single_frame_True.csv
/Users/k1801626/OneDrive - King's College London/data/JP/AFT_track/siCCT8_si5/siCCT8_si5_Spots_AFT_single_frame_True.csv


In [18]:
tracks_list = glob.glob(csv_folder+'*Tracks.csv')

print(*tracks_list, sep="\n")

/Users/k1801626/OneDrive - King's College London/data/JP/AFT_track/merged_Tracks.csv


In [19]:
df_tracks = pd.read_csv(tracks_list[0])
df_tracks.head()

,LABEL,TRACK_INDEX,TRACK_ID,DIVISION_TIME_MEAN,DIVISION_TIME_STD,NUMBER_SPOTS,NUMBER_GAPS,NUMBER_SPLITS,NUMBER_MERGES,NUMBER_COMPLEX,...,CONFINEMENT_RATIO,MEAN_STRAIGHT_LINE_SPEED,LINEARITY_OF_FORWARD_PROGRESSION,MEAN_DIRECTIONAL_CHANGE_RATE,File_name_raw,Condition,experiment_nb,File_name,Repeat,Unique_ID
0,Track_1,1,1,NaN,NaN,76,0,0,0,0,...,0.651187,10.556033,0.651187,1.235458,Position_26,siCCT8_si5,R1,siCCT8_si5_R1_Position_26,1,siCCT8_si5_R1_Position_26_1
1,Track_2,2,2,NaN,NaN,55,0,0,0,0,...,0.013550,0.361223,0.013550,1.681933,Position_26,siCCT8_si5,R1,siCCT8_si5_R1_Position_26,1,siCCT8_si5_R1_Position_26_2
2,Track_3,3,3,NaN,NaN,76,0,0,0,0,...,0.149645,1.959934,0.149645,1.803235,Position_26,siCCT8_si5,R1,siCCT8_si5_R1_Position_26,1,siCCT8_si5_R1_Position_26_3
3,Track_4,4,4,NaN,NaN,76,0,0,0,0,...,0.044778,0.713082,0.044778,1.874086,Position_26,siCCT8_si5,R1,siCCT8_si5_R1_Position_26,1,siCCT8_si5_R1_Position_26_4
4,Track_8,8,8,NaN,NaN,29,0,0,0,0,...,0.140809,2.646769,0.140809,1.866365,Position_26,siCCT8_si5,R1,siCCT8_si5_R1_Position_26,1,siCCT8_si5_R1_Position_26_8


### Save merged spots df (with AFT track results)

In [11]:
df_all = pd.DataFrame()

for df in range(len(file_list)):
    df_temp = pd.read_csv(file_list[df])

    df_list = [df_all, df_temp]
    df_all = pd.concat([df_all for df_all in df_list if not df_all.empty])

In [21]:
file_name = 'merged_Spots_AFT_single_frame_'+str(single_frame)+'.csv'

df_all.to_csv(csv_folder+file_name)

### Average angle correlation per track

In [15]:
av_cos2 = []
for uniqueID in range(len(df_all.Unique_ID.unique())):
    df_temp = df_all.loc[df_all.Unique_ID == df_all.Unique_ID.unique()[uniqueID]]
    av_cos2 = np.append(av_cos2, df_temp.AFT_track_angle_cos2.mean())
av_cos2

array([0.57944967, 0.26335979, 0.47335806, 0.36732236, 0.19864387,
       0.39094669, 0.4129381 , 0.30119748, 0.40832608, 0.57784819,
       0.4664177 , 0.55766448, 0.51104464, 0.50153187, 0.41362916,
       0.42706723, 0.20114336, 0.37778121, 0.31378601,        nan,
       0.32796136,        nan, 0.42495413, 0.40545387, 0.38319751,
       0.45887137, 0.48862917])

In [24]:
df_subset_out_tracks = pd.DataFrame(columns = ['Unique_ID','AFT_tracks_cos2_mean'])

df_subset_out_tracks.Unique_ID = df_all.Unique_ID.unique()
df_subset_out_tracks.AFT_tracks_cos2_mean = av_cos2

In [25]:
df_subset_out_tracks.head()

,Unique_ID,AFT_tracks_cos2_mean
0,siCCT8_si8_R1_Position_31_0,0.579450
1,siCCT8_si8_R1_Position_31_1,0.263360
2,siCCT8_si8_R1_Position_31_2,0.473358
3,siCCT8_si8_R1_Position_31_3,0.367322
4,siCCT8_si8_R1_Position_31_4,0.198644


### Save merged tracks df (with AFT track results)

In [20]:
df_tracks_out = pd.merge(df_tracks, df_subset_out_tracks, on='Unique_ID', how='outer')

df_tracks_out.head()                     

,LABEL,TRACK_INDEX,TRACK_ID,DIVISION_TIME_MEAN,DIVISION_TIME_STD,NUMBER_SPOTS,NUMBER_GAPS,NUMBER_SPLITS,NUMBER_MERGES,NUMBER_COMPLEX,...,MEAN_STRAIGHT_LINE_SPEED,LINEARITY_OF_FORWARD_PROGRESSION,MEAN_DIRECTIONAL_CHANGE_RATE,File_name_raw,Condition,experiment_nb,File_name,Repeat,Unique_ID,AFT_tracks_cos2_mean
0,Track_1,1.0,1.0,NaN,NaN,76.0,0.0,0.0,0.0,0.0,...,10.556033,0.651187,1.235458,Position_26,siCCT8_si5,R1,siCCT8_si5_R1_Position_26,1.0,siCCT8_si5_R1_Position_26_1,0.427067
1,Track_2,2.0,2.0,NaN,NaN,55.0,0.0,0.0,0.0,0.0,...,0.361223,0.013550,1.681933,Position_26,siCCT8_si5,R1,siCCT8_si5_R1_Position_26,1.0,siCCT8_si5_R1_Position_26_2,0.201143
2,Track_3,3.0,3.0,NaN,NaN,76.0,0.0,0.0,0.0,0.0,...,1.959934,0.149645,1.803235,Position_26,siCCT8_si5,R1,siCCT8_si5_R1_Position_26,1.0,siCCT8_si5_R1_Position_26_3,0.377781
3,Track_4,4.0,4.0,NaN,NaN,76.0,0.0,0.0,0.0,0.0,...,0.713082,0.044778,1.874086,Position_26,siCCT8_si5,R1,siCCT8_si5_R1_Position_26,1.0,siCCT8_si5_R1_Position_26_4,0.313786
4,Track_8,8.0,8.0,NaN,NaN,29.0,0.0,0.0,0.0,0.0,...,2.646769,0.140809,1.866365,Position_26,siCCT8_si5,R1,siCCT8_si5_R1_Position_26,1.0,siCCT8_si5_R1_Position_26_8,0.327961


In [23]:
file_name = 'merged_Tracks_AFT_single_frame_'+str(single_frame)+'.csv'
df_tracks_out.to_csv(csv_folder+file_name)